In [19]:
from elections.DefaultConfigOptions import unit_election_config
from elections.Ballot import Ballot
from elections.ElectionConstructor import ElectionConstructor, construct_irv
from elections.Ideology import Ideology
from elections.PopulationGroup import Independents
from elections.Candidate import Candidate
from elections.NDPopulation import NDPopulation
from network import ElectionModel
from typing import List
from network.Tensor import Tensor
import numpy as np
import tensorflow as tf
import random
import math

n_bins = 21
max_ideology = 1.5
min_ideology = -1.5
ideology_range = max_ideology - min_ideology
ideology_dim = 1
ideology_zone = .6
n_voters = 100

In [20]:
def gen_random_candidates(population: NDPopulation, n: int)-> List[Candidate]:
    candidates = []
    for i in range(n):
        ivec = population.unit_sample_voter().ideology.vec * .5
        candidates.append(Candidate(f"c-{i}", Independents, Ideology(ivec), 0))

    return candidates

def convert_bin_to_ideology(bin: int) -> float:
    lower = bin / n_bins * ideology_range + min_ideology
    upper = (bin + 1) / n_bins * ideology_range + min_ideology
    return random.uniform(lower, upper)

def convert_ideology_to_bin(ideology: float) -> int:
    ideology = max(min_ideology, min(ideology, max_ideology))
    pct = (ideology - min_ideology) / ideology_range
    return int(pct * n_bins)

# note that the return value is the index of the winning candidate and NOT the
# bin of the winning candidate.
def convert_candidates_to_input_vec( candidates: List[Candidate]) -> Tensor:
    cc = [convert_ideology_to_bin(c.ideology.vec[0]) for c in candidates]
    x = np.zeros(shape=(1, n_bins), dtype=np.single)
    for c in cc:
        x[0, c] = 1
    return x

def create_training_sample(candidates: List[Candidate], winner: Candidate) -> (int, list[int]):
    w = candidates.index(winner)
    cc = [convert_ideology_to_bin(c.ideology.vec[0]) for c in candidates]
    return cc, w

class ExtendedCandidate:
    def __init__(self, base_candidate: Candidate):
        self.base_candidate = base_candidate
        self.current_bin = convert_ideology_to_bin(base_candidate.ideology.vec[0])
        self.current_candidate = base_candidate

    def win_bonus(self, ideology: float) -> float:
        delta = math.fabs(ideology - self.base_candidate.ideology.vec[0])
        return 1 - min(delta / ideology_zone, 1.0)

    def best_position(self, model: ElectionModel, other_candidates: List[Candidate], bin_range: int) -> float:
        x = convert_candidates_to_input_vec(other_candidates)
        win_probabilities = model(x).numpy()

        best_return = -1
        best_ideology = -1
        b_start = self.current_bin - bin_range
        b_end =  min(n_bins, self.current_bin + bin_range + 1)
        for b in range(b_start, b_end):
            ideology = convert_bin_to_ideology(b)
            wb = self.win_bonus(ideology)
            expected_return = wb * win_probabilities[0, b]
            if expected_return > best_return:
                best_return = expected_return
                best_ideology = ideology

        return best_ideology

    def best_candidate(self, model: ElectionModel, all_candidates: List[Candidate], bin_range: int):
        other_candidates = list(filter(lambda c: c != self.current_candidate, all_candidates))
        assert(len(other_candidates) == 4)
        ideology = self.best_position(model, other_candidates, bin_range)
        self.current_bin = convert_ideology_to_bin(ideology)

        self.current_candidate = Candidate(
            self.base_candidate.name,
            self.base_candidate.party,
            Ideology(np.array([ideology])),
            self.base_candidate.quality)

        return self.current_candidate

def create_population() ->  NDPopulation:
    population_means = np.zeros(shape=(1,))
    population_stddev = np.ones(shape=(1,))
    return NDPopulation(population_means, population_stddev)

In [21]:
def run_sample_election( candidates: List[Candidate],
                         process: ElectionConstructor,
                         population: NDPopulation,
                         n_voters: int) -> Candidate:

    print("run_sample_election")
    for c in candidates:
        print(f"\tcandidate {c}")

    voters = population.generate_unit_voters(n_voters)
    ballots = [Ballot(v, candidates, unit_election_config) for v in voters]
    result = process.run(ballots, set(candidates))
    winner = result.winner()
    return winner

In [22]:
def run_strategic_races():
    population = create_population()
    model = tf.keras.models.load_model("networks/net.2048.512.3.mdl")
    process = ElectionConstructor(construct_irv, "IRV")

    for i in range(10):
        candidates = gen_random_candidates(population, 5)
        extended_candidates = [ExtendedCandidate(c) for c in candidates]
        for bb in [1, 2, 3]:
            cc = [ec.current_candidate for ec in extended_candidates]
            new_candidates = [e.best_candidate(model, cc, bb) for e in extended_candidates]
            candidates = new_candidates

        w = run_sample_election(candidates, process, population, 1000)
        print(f"w.ideology: {w.ideology}")

run_strategic_races()

creating candidate with ideology [0.71592529]
creating candidate with ideology [-0.10068364]
creating candidate with ideology [-0.0014854]
creating candidate with ideology [0.21776586]
creating candidate with ideology [-1.1878918]
creating candidate with ideology [0.63565218]
creating candidate with ideology [-0.05622496]
creating candidate with ideology [-0.12576821]
creating candidate with ideology [0.20238829]
creating candidate with ideology [-0.97595767]
creating candidate with ideology [0.62096035]
creating candidate with ideology [0.02937786]
creating candidate with ideology [-0.24128693]
creating candidate with ideology [-0.00557224]
creating candidate with ideology [-0.76098355]
creating candidate with ideology [0.47670547]
creating candidate with ideology [-0.35665496]
creating candidate with ideology [-0.11873689]
creating candidate with ideology [0.45480061]
creating candidate with ideology [-0.89357452]
run_sample_election
	candidate <elections.Candidate.Candidate object a

In [23]:
location=0
scale=1
ideology = np.random.normal(loc=location, scale=scale)
ideology = np.array([ideology])
print(f"np.random.normal returns {ideology}")
print(np.random.normal(0, 1))

np.random.normal returns [0.707714]
-0.18489844629000776
